# Classificação de risco de crédito otimizando os modelos Ensemble com Randomized Search (Parte 2) 

### Objetivo
Nesta segunda etapa, o objetivo será avaliar o quanto a acurácia dos modelos de classificação com **métodos ensemble** melhorará fazendo ajustes com o Randomized Search nos hiperparâmetros dos algortimos selecionados.

### Algoritmos utilizados na Parte 1 que serão testados nesta etapa:
- 1º Gradient Boosting: acurária anterior de 82,03%
- 2º Random Forest = acurácia anterior de 81,64%
- 3º Extra Trees (Extremely Randomized Forest) = acurácia anterior de 81,25%


## Pacotes e funções

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as mpl
from matplotlib import pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## Base de dados

In [2]:
credit_data = pd.read_excel('credit.xls', skiprows = 1)

In [3]:
# Variáveis da base de dados
print(credit_data.columns)

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')


In [4]:
# Quantidades de observações e de variáveis
print(credit_data.shape)

(30000, 25)


**Variável-target**

In [5]:
target = 'default payment next month'
y = np.asarray(credit_data[target])

**Variáveis explicativas**

In [6]:
features = credit_data.columns.drop(['ID', target])
X = np.asarray(credit_data[features])

**Bases para treino (70%) e para teste (30%)**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 10)

# Modelo 1 - Random Forest

In [8]:
# Parâmetros a serem testados:
param_dist = {"n_estimators": [10, 20, 50],
              "max_depth": [1, 4, 6, 8],
              "max_features": [10, 16, 20],
              "min_samples_split": [2, 8, 16, 18],
              "min_samples_leaf": [1, 2, 3, 4, 6, 8],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Treinamento do modelo:
rs_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions = param_dist, n_iter = 25, return_train_score = True)
rs_search.fit(X_train, y_train)

# Melhor estimador:
bestrs = rs_search.best_estimator_
print (bestrs)

RandomForestClassifier(max_depth=4, max_features=16, min_samples_leaf=2,
                       min_samples_split=8, n_estimators=10)


**Comentário:** dentre os parâmetros apresentados ao novo modelo, o melhor foi construído com as configurações acima.

In [9]:
# Predição usando melhor o estimador:
rs_pred = bestrs.predict(X_test)
print("Acurácia em treino = ", rs_search.score(X_train, y_train))
print("Acurácia em teste = ", accuracy_score(y_test, rs_pred))
print(confusion_matrix(y_test, rs_pred))

Acurácia em treino =  0.8238095238095238
Acurácia em teste =  0.8211111111111111
[[6684  298]
 [1312  706]]


# Modelo 2 - Extra Trees

In [10]:
# Parâmetros a serem testados:
param_dist_et = {"n_estimators": [10, 20, 50],
                 "max_depth": [1, 4, 6, 8],
                 "max_features": [10, 16, 20],
                 "min_samples_split": [2, 8, 16, 18],
                 "min_samples_leaf": [1, 2, 3, 4, 6, 8],
                 "bootstrap": [True, False],
                 "criterion": ["gini", "entropy"]}

# Combinando parâmetros:
rs_extrees = RandomizedSearchCV(ExtraTreesClassifier(), param_distributions = param_dist_et, n_iter = 25, return_train_score = True)  
rs_extrees.fit(X_train, y_train)

# Melhor estimador
bestrset = rs_extrees.best_estimator_
print (bestrset)

ExtraTreesClassifier(max_depth=6, max_features=20, min_samples_split=16,
                     n_estimators=20)


In [11]:
# Nova predição
rs_et_pred = bestrset.predict(X_test)
print("Acurácia em treino = ", rs_extrees.score(X_train, y_train))
print("Acurácia em teste = ", accuracy_score(y_test, rs_et_pred))
print(confusion_matrix(y_test, rs_et_pred))

Acurácia em treino =  0.8252380952380952
Acurácia em teste =  0.8212222222222222
[[6665  317]
 [1292  726]]


# Modelo 3 - Gradient Boosting

In [12]:
# Parâmetros a serem testados
param_dist_gb = {"n_estimators": [10, 20, 50],
                 "max_depth": [1, 4, 6, 8],
                 "max_features": [10, 16, 20],
                 "min_samples_split": [2, 8, 16, 18],
                 "min_samples_leaf": [1, 2, 3, 4, 6, 8],
                 "learning_rate": [1.0, 0.1, 0.01]}

# Combinando parâmetros
rs_gb = RandomizedSearchCV(GradientBoostingClassifier(), param_distributions = param_dist_gb, n_iter = 25, return_train_score = True)  
rs_gb.fit(X_train, y_train)

# Melhor estimador
bestrsgb = rs_gb.best_estimator_
print (bestrsgb)

GradientBoostingClassifier(max_depth=4, max_features=20, min_samples_leaf=8,
                           min_samples_split=8, n_estimators=20)


In [13]:
# Nova predição
rs_gb_pred = bestrsgb.predict(X_test)
print("Acurácia em treino = ", rs_gb.score(X_train, y_train))
print("Acurácia em teste = ", accuracy_score(y_test, rs_gb_pred))
print(confusion_matrix(y_test, rs_gb_pred))

Acurácia em treino =  0.8257619047619048
Acurácia em teste =  0.8207777777777778
[[6695  287]
 [1326  692]]


### Resultados:


As otimizações obtidas usando os parâmetros e hiperparâmetros selecionados através do Randomized Search foram positivas, mas bem discretas:  
- Random Forest = a acurácia passou de 81,64% para 82,11% (diferença de 0,47%).
- Extra Trees = a acurácia passou de 81,25% para 82,12% (diferença de 0,87%).
- Gradient Boosting: a acurária passou de 82,03% para 82,07% (diferença de 0,04%).
Nessa etapa, o melhor resultado foi obtido com o **Extra Trees**: 82,12% de acurária, com incremento de 0,87%.